In [1]:
import os
os.chdir("..")
os.getcwd()

'/Users/Louis/ml_projects/ToxPost'

In [ ]:
import src.features.preprocess as preprocess

..loading the data..


  0%|          | 81/159570 [00:00<03:28, 765.21it/s]

..using NLP to clean each comment in the corpus.. 


100%|██████████| 159570/159570 [02:45<00:00, 966.02it/s] 


..writing the results to the cleaned data file
..using TfIdf to shrink each comment in the corpus to size 100..


...

..loading the Glove embedding..


1193515it [00:18, 62906.18it/s]


using PCA to reduce the embedding space of each word to size 20


100%|██████████| 159570/159570 [00:07<00:00, 20066.01it/s]


..writing the results to the embedded data file..


In [5]:
import src.features.preprocess as prep

In [6]:
prep.py

AttributeError: module 'src.features.preprocess' has no attribute 'py'

In [7]:
prep.preprocess()

..loading the data..


  0%|          | 149/159570 [00:00<03:31, 753.05it/s]

..using NLP to clean each comment in the corpus.. 


100%|██████████| 159570/159570 [02:59<00:00, 887.41it/s] 


..using TfIdf to shrink each comment in the corpus to size 100..


100%|██████████| 159570/159570 [04:03<00:00, 656.66it/s]


KeyboardInterrupt: 

In [ ]:
raw_training_data = "./data/raw/train.csv"

data = load_corpus(raw_training_data, header=True, id=True)

In [4]:
features = [datapoint[0] for datapoint in data]
labels = [label[1] for label in data]

In [5]:
d = {"l":1, "t":2}

In [7]:
[values for values in d.values()]

[1, 2]

just to give an example:

In [5]:
print(data[105])

[['I', 'tend', 'to', 'think', 'that', 'when', 'the', 'list', 'is', 'longer', 'than', 'the', 'rest', 'of', 'the', 'article,', "there's", 'a', 'problem.', 'Either', 'the', 'history', 'and', 'characteristics', 'should', 'be', 'expanded,', 'of', 'the', 'list', 'should', 'be', 'culled.', 'I', 'personally', 'like', 'the', 'consensus', 'that', 'was', 'reached', 'in', 'the', 'Tripel', 'article', 'between', 'and', 'others...myself', 'included.', 'We', 'decided', 'that', 'the', 'country', 'of', 'origin', 'should', 'be', 'highlighted', 'with', 'the', 'most', 'examples.', 'There', 'we', 'did', 'Trappist,', 'Abbey,', 'and', 'Foreign.', 'Foreign', 'includes', 'all', 'non-Belgium', 'examples.', 'This', 'way', 'the', 'article', 'is', 'an', 'article', 'in', 'an', 'encyclopedia,', 'not', 'a', 'list.', 'People', 'can', 'find', 'lists', 'anywhere,', 'and', 'the', 'article', 'should', 'reference', 'this.', 'Also,', 'the', 'style', 'statistics', 'are', 'bad.', 'When', 'you', 'say', 'style', 'statistics,', '

In [6]:
features[105]

['I',
 'tend',
 'to',
 'think',
 'that',
 'when',
 'the',
 'list',
 'is',
 'longer',
 'than',
 'the',
 'rest',
 'of',
 'the',
 'article,',
 "there's",
 'a',
 'problem.',
 'Either',
 'the',
 'history',
 'and',
 'characteristics',
 'should',
 'be',
 'expanded,',
 'of',
 'the',
 'list',
 'should',
 'be',
 'culled.',
 'I',
 'personally',
 'like',
 'the',
 'consensus',
 'that',
 'was',
 'reached',
 'in',
 'the',
 'Tripel',
 'article',
 'between',
 'and',
 'others...myself',
 'included.',
 'We',
 'decided',
 'that',
 'the',
 'country',
 'of',
 'origin',
 'should',
 'be',
 'highlighted',
 'with',
 'the',
 'most',
 'examples.',
 'There',
 'we',
 'did',
 'Trappist,',
 'Abbey,',
 'and',
 'Foreign.',
 'Foreign',
 'includes',
 'all',
 'non-Belgium',
 'examples.',
 'This',
 'way',
 'the',
 'article',
 'is',
 'an',
 'article',
 'in',
 'an',
 'encyclopedia,',
 'not',
 'a',
 'list.',
 'People',
 'can',
 'find',
 'lists',
 'anywhere,',
 'and',
 'the',
 'article',
 'should',
 'reference',
 'this.',
 'Al

In [7]:
clean_corpus([features[105]])

100%|██████████| 1/1 [00:00<00:00, 144.13it/s]


[['tend',
  'think',
  'list',
  'longer',
  'rest',
  'article',
  'theres',
  'problem',
  'either',
  'history',
  'characteristics',
  'expanded',
  'list',
  'culled',
  'personally',
  'like',
  'consensus',
  'reached',
  'tripel',
  'article',
  'othersmyself',
  'included',
  'decided',
  'country',
  'origin',
  'highlighted',
  'examples',
  'trappist',
  'abbey',
  'foreign',
  'foreign',
  'includes',
  'nonbelgium',
  'examples',
  'way',
  'article',
  'article',
  'encyclopedia',
  'list',
  'people',
  'find',
  'lists',
  'anywhere',
  'article',
  'reference',
  'also',
  'style',
  'statistics',
  'bad',
  'say',
  'style',
  'statistics',
  'sounds',
  'like',
  'prescription',
  'description',
  'encyclopedia',
  'follow',
  'latter',
  'former',
  'used',
  'citation',
  'followand',
  'youll',
  'find',
  'valid',
  'one',
  'style',
  'country',
  'origin']]

In [8]:
features[0]

("D'aww!",
 'He',
 'matches',
 'this',
 'background',
 'colour',
 "I'm",
 'seemingly',
 'stuck',
 'with.',
 'Thanks.',
 '(talk)',
 '21:51,',
 'January',
 '11,',
 '2016',
 '(UTC)')

  2%|▏         | 3464/159570 [00:20<03:09, 824.35it/s]

In [11]:
print(features[105])

('I', 'tend', 'to', 'think', 'that', 'when', 'the', 'list', 'is', 'longer', 'than', 'the', 'rest', 'of', 'the', 'article,', "there's", 'a', 'problem.', 'Either', 'the', 'history', 'and', 'characteristics', 'should', 'be', 'expanded,', 'of', 'the', 'list', 'should', 'be', 'culled.', 'I', 'personally', 'like', 'the', 'consensus', 'that', 'was', 'reached', 'in', 'the', 'Tripel', 'article', 'between', 'and', 'others...myself', 'included.', 'We', 'decided', 'that', 'the', 'country', 'of', 'origin', 'should', 'be', 'highlighted', 'with', 'the', 'most', 'examples.', 'There', 'we', 'did', 'Trappist,', 'Abbey,', 'and', 'Foreign.', 'Foreign', 'includes', 'all', 'non-Belgium', 'examples.', 'This', 'way', 'the', 'article', 'is', 'an', 'article', 'in', 'an', 'encyclopedia,', 'not', 'a', 'list.', 'People', 'can', 'find', 'lists', 'anywhere,', 'and', 'the', 'article', 'should', 'reference', 'this.', 'Also,', 'the', 'style', 'statistics', 'are', 'bad.', 'When', 'you', 'say', 'style', 'statistics,', 'i

In [7]:
cleaned_features = [clean_document(feature) for feature in tqdm.tqdm(features)]

100%|██████████| 159570/159570 [03:00<00:00, 885.03it/s] 


In [9]:
import  csv

with open("out.csv","w") as f:
    wr = csv.writer(f,delimiter="\n")
    wr.writerow(cleaned_features)

In [8]:
shrunken_corpus = shrink_corpus(cleaned_features, 100)

  0%|          | 120/159570 [00:00<02:13, 1195.18it/s]

TfIdf scores calculated...



100%|██████████| 159570/159570 [02:13<00:00, 1197.47it/s]


In [1]:
glove = "./recourses/glove/glove.twitter.27B.25d.txt"

In [7]:
embedding, embeddable_words = load_embedding(glove)

1193515it [00:22, 53302.95it/s]


Embedding loaded... 



In [8]:
preprocess()

159571it [00:06, 23748.30it/s]



done reading ./data/raw/train.csv


100%|██████████| 159570/159570 [02:49<00:00, 943.39it/s] 


In [9]:
"Trappis,".translate(str.maketrans('', '', string.punctuation))

'Trappis'

In [8]:
import string

In [ ]:
l = ["a","b"]

In [4]:
d = {'1': 1, "2": 2}

In [6]:
[values for values in d.values()]

[1, 2]

In [5]:
d.get("3")

In [24]:
la = [1,2,3]
lb = [5,6,7]

In [21]:
l = list(zip(la, lb))

In [23]:
l

[(1, 5), (2, 6), (3, 7)]

In [26]:
la.index(3)

2

In [4]:
import numpy as np

In [5]:
a = np.array([1,2,3])

In [6]:
a.reshape(-1,1)

array([[1],
       [2],
       [3]])

In [11]:
la = [1,2, 3]

In [12]:
lb = [4, 5, 6]

In [23]:
l =[list(item) for item in zip(la, lb)]

In [24]:
print(l)

[[1, 4], [2, 5], [3, 6]]


In [9]:
l = [[a,b] for a in la for b in lb ]

In [10]:
l

[[1, 4], [1, 5], [1, 6], [2, 4], [2, 5], [2, 6], [3, 4], [3, 5], [3, 6]]